In [1]:
import sympy as smp
import numpy as np
from sympy.physics.quantum.dagger import Dagger
from sympy.physics.quantum import TensorProduct as TP

In [239]:
def hs(mat1, mat2):
    return smp.N(np.trace(Dagger(mat1)*mat2))

def norm(mat1):
    return mat1/hs(mat1, mat1)

In [3]:
I = smp.Matrix([[1,0,0], [0,1,0], [0,0,1]])
X = smp.Matrix([[0,0,1],[1,0,0], [0,1,0]])
Z = smp.Matrix([[1,0,0], [0,smp.E**(1j*2*smp.pi/3),0], [0, 0, smp.E**(1j*4*smp.pi/3)]])


H = (1/smp.sqrt(3))*smp.Matrix([[1,1,1],[1,smp.E**(1j*2*smp.pi/3),smp.E**(1j*4*smp.pi/3)], [1, smp.E**(1j*4*smp.pi/3), smp.E**(1j*2*smp.pi/3)]])
#S = smp.Matrix([[1,0,0], [0,smp.E**(1j*2*smp.pi/3),0], [0, 0, 1]])#smp.sqrt(Z).doit().factor(deep=True)
S = smp.Matrix([[smp.E**(1j*2*smp.pi/3),0,0], [0,1,0], [0, 0, 1]])
#Y=Z*X
CZ = TP(smp.Matrix([[1,0, 0], [0,0, 0], [0, 0, 0]]), I)+TP(smp.Matrix([[0,0, 0], [0,1, 0], [0, 0, 0]]), Z) + TP(smp.Matrix([[0,0, 0], [0,0, 0], [0, 0, 1]]), Z**2)
CX = TP(smp.Matrix([[1,0, 0], [0,0, 0], [0, 0, 0]]), I)+TP(smp.Matrix([[0,0, 0], [0,1, 0], [0, 0, 0]]), X) + TP(smp.Matrix([[0,0, 0], [0,0, 0], [0, 0, 1]]), X**2)
HS = smp.Matrix(H*S) #S, Z, I

T_2 = TP(HS, HS)*CZ
T_3 = TP(HS, HS, HS)*TP(CZ, I)*TP(I, CZ)
T_4 = TP(HS, HS, HS, HS)*TP(CZ, CZ)*TP(I,CZ,I)

w = smp.N(smp.E**(2*smp.I*smp.pi*1/3))

In [150]:
Z_first = TP(Z, I, I) + TP(Z**2, I, I)

Z_second = TP(Z, I, I) + w*TP(Z**2, I, I)

smp.N(hs(Z_first, Z_second))

27.0000000000000

In [153]:
smp.N(hs(Z_first, Z_first))

54.0000000000000

In [156]:
Z_1_ortho = smp.N(norm(Z_first))

In [157]:
Z_2_ortho = smp.N(norm(Z_second))

In [189]:
Z_3 = smp.N(Z_2_ortho\
            -(1.2*Z_1_ortho))

In [190]:
smp.N(hs(Z_1_ortho, Z_3))

1.04403065089105

In [240]:
I = smp.Matrix([[1,0,0], [0,1,0], [0,0,1]])
X = smp.Matrix([[0,0,1],[1,0,0], [0,1,0]])

In [251]:
first = norm(1.5*I+2*X)

In [252]:
first

Matrix([
[             0.08,                 0, 0.106666666666667],
[0.106666666666667,              0.08,                 0],
[                0, 0.106666666666667,              0.08]])

In [253]:
np.trace(1.5*I+2*X)

4.50000000000000

In [289]:
first = Z*X+Z*Z
second = Z*X+w*Z*Z

In [290]:
first = first/smp.sqrt(smp.Abs(hs(first, first)))

In [294]:
third = smp.N(second - hs(first, second)*first)

In [1]:
dim = 3 # Dimension

def modadd(state, Z_or_X, index):
    if state[Z_or_X][index] == dim-1:
        state[Z_or_X][index]=0
    else:
        state[Z_or_X][index]+=1
    return state

def evolution(state, size):
    Z_part, X_part = state
    new_state = [[0 for _ in range(size)],[0 for _ in range(size)]]
    # X to Z evolution
    for i in range(size):
        for k in range(X_part[i]):
            # Z
            new_state = modadd(new_state, 0, i)
            new_state = modadd(new_state, 0, i)
    # Z evolution
    for i in range(size):
        for k in range(Z_part[i]):
            if i == 0: # Boundary
                # Z
                new_state = modadd(new_state, 0, i)
                new_state = modadd(new_state, 0, i)
                new_state = modadd(new_state, 0, i+1)
                new_state = modadd(new_state, 0, i+1)
                # X
                new_state = modadd(new_state, 1, i)
            elif i < size-1:
                # Z
                new_state = modadd(new_state, 0, i-1)
                new_state = modadd(new_state, 0, i-1)
                new_state = modadd(new_state, 0, i)
                new_state = modadd(new_state, 0, i)
                new_state = modadd(new_state, 0, i+1)
                new_state = modadd(new_state, 0, i+1)
                # X
                new_state = modadd(new_state, 1, i)
            else:
                # Z
                new_state = modadd(new_state, 0, i-1)
                new_state = modadd(new_state, 0, i-1)
                new_state = modadd(new_state, 0, i)
                new_state = modadd(new_state, 0, i)
                # X
                new_state = modadd(new_state, 1, i) 
    return new_state

state_0=[[1, 0, 0],[0, 0, 0]] # (Z, X)

size = len(state_0[0])

state = evolution(state_0, size)

all_states = [state_0]

while state != state_0:
    all_states.append(state)
    state = evolution(state, size)
print(len(all_states))


30


In [2]:
l3 = np.matrix([[1, 0, 0], [0, -1, 0], [0, 0, 0]])

l8 = np.matrix([[1, 0, 0], [0, 1, 0], [0, 0, -2]])*1/np.sqrt(3)


NameError: name 'np' is not defined